In [1]:
#All libraries that will be used thorughout this project
import pandas as pd
import numpy as np
import seaborn as sbs
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split

In [2]:
# Import Data that will be used for analysis

#Shiba data
shiba_df = pd.read_csv("D:\Portfolio Projects\Python Stock Analysis\Coins Data\SHIB-USD.csv")

#Doge data
doge_df = pd.read_csv("D:\Portfolio Projects\Python Stock Analysis\Coins Data\DOGE-USD.csv")

In [3]:
# After examination a minimum amount of data cleaning was needed.

#Shiba - Need to drop lines with $0 dlls value on 'Adj Close' and NaNs 
shiba_df = shiba_df[shiba_df.Close != 0]
shiba_df = shiba_df.dropna()
shiba_df = shiba_df.reset_index(drop = True)

#Doge - Need to drop lines with NaNs
doge_df = doge_df.dropna()
doge_df = doge_df.reset_index(drop = True)

In [4]:
# For this Analysis we will be focusing on 'Adj Close' along with the Dates

#Creating a variable for the Adj Close Amount for each Coin
shiba_close = shiba_df[['Date','Adj Close']]

doge_close = doge_df[['Date','Adj Close']]


In [5]:
# Here we will be leaving variables that we will be using throughout the program 
# you can adjust them at will to change the analysis we will be doing

####################################
##Prediction of Days in the Future##
####################################

Future_Prediciton = 10

In [6]:
# The first approach will be to create a linear regresion to predict future prices
# Let's create data frames for this first approach that way we will always have our previous data frames ready to be used in different models

shiba_linear = shiba_close
doge_linear = doge_close




In [7]:
# Lets creat new columns on each data frame and lets call them "Future Prediction"
# on this new column we are going to use shift (and our targeted days to predict)
shiba_linear['Future Prediction'] = shiba_linear[['Adj Close']].shift(-Future_Prediciton)
doge_linear['Future Prediction'] = doge_linear[['Adj Close']].shift(-Future_Prediciton)

<ipython-input-7-4988c45e1912>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shiba_linear['Future Prediction'] = shiba_linear[['Adj Close']].shift(-Future_Prediciton)
<ipython-input-7-4988c45e1912>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  doge_linear['Future Prediction'] = doge_linear[['Adj Close']].shift(-Future_Prediciton)


In [8]:
# Now we will creat the indepedent (x) data sets for each coin
shiba_x = np.array(shiba_close.drop(['Date','Future Prediction'],1))
doge_x = np.array(doge_close.drop(['Date', 'Future Prediction'],1))

# Next we want to remove from this set the number of days that we want to forecast (this is how we will test the model)
shiba_x = shiba_x[:-Future_Prediciton]
doge_x = doge_x[:-Future_Prediciton]

In [9]:
# Now we will create the dependent data frame (y) which will be coming fomr th "Future Prediciton" Column
shiba_y = np.array(shiba_close.drop(['Date','Adj Close'],1))
doge_y = np.array(doge_close.drop(['Date','Adj Close'],1))

# We also want to remove the forecasted days from this set
shiba_y = shiba_y[:-Future_Prediciton]
doge_y = doge_y[:-Future_Prediciton]


In [10]:
# Here we will set the traning and testing percentages that will be used in our Linear Regresion model

# For Shiba
shiba_x_train, shiba_x_test, shiba_y_train, shiba_y_test = train_test_split(shiba_x, shiba_y, test_size=0.2)

# For Doge
doge_x_train, doge_x_test, doge_y_train, doge_y_test = train_test_split(doge_x, doge_y, test_size=0.2)

In [11]:
# Now we create and train the SVM regressor for each coin

#Shiba
shiba_svr_rbf = SVR(kernel='rbf', C=1e3, gamma=0.1)
shiba_svr_rbf.fit(shiba_x_train, np.ravel(shiba_y_train))

#Doge
doge_svr_rbf = SVR(kernel='rbf', C=1e3, gamma=0.1)
doge_svr_rbf.fit(doge_x_train, np.ravel(doge_y_train))


SVR(C=1000.0, gamma=0.1)

In [12]:
# Here we will test the confidence level on the previous calculations
# For reference the best number we can get is 1.00

shiba_svm_confidence = shiba_svr_rbf.score(shiba_x_test, shiba_y_test)
print("Shiba confidence is : ", shiba_svm_confidence)

doge_svm_confidence = doge_svr_rbf.score(doge_x_test, doge_y_test)
print("Doge confidence is : ", doge_svm_confidence)

Shiba confidence is :  -1.3831598199210489
Doge confidence is :  0.27599284904180976


From the results above you can see that both coins are pretty hard to group/lable the prices or "learn" from them. 
The negative confidence in Shiba means that the program did not learn anything from the data set
Even though Doge is considerably low it demonstrated to be a bit easier to "learn" from the data provided. 

This numbers would be different if they would be looking at steady numbers like regular stocks. Or any assets that is not as volatile as Cryptocurrency at the moment. SVM seems to be unefficient trying to group prices for these two coins.

Nevertheless we will still try to apply the regression model on both coins. 

In [13]:
# Here we will create the Liner Regresion for Shiba Inu

shiba_lr = LinearRegression()

# Now we will trian the model 

shiba_lr.fit(shiba_x_train, shiba_y_train)

# Now we will test the confidence score
shiba_lr_confidence = shiba_lr.score(shiba_x_test, shiba_y_test)
print("Shiba Confidence Score is : ", shiba_lr_confidence)


Shiba Confidence Score is :  0.7346921662609301


In [14]:
# Here we will create the Liner Regresion for Doge

doge_lr = LinearRegression()

# Now we will trian the model 

doge_lr.fit(doge_x_train, doge_y_train)

# Now we will test the confidence score
doge_lr_confidence = doge_lr.score(doge_x_test, doge_y_test)
print("Doge Confidence Score is : ", doge_lr_confidence)

Doge Confidence Score is :  0.8884959261021148


It looks like Linear Regression is going to be our best bet to analyze these coins.

Let's go ahead and try to predict future prices for each coin using linear regresion

In [16]:
# Here we are only going to keep the amount of days we want to predict focusing on Adj Close Price

#Shiba
shiba_forecast = np.array(shiba_close.drop(['Date', 'Future Prediction'],1))[-Future_Prediciton:]

#Doge
doge_forecast = np.array(doge_close.drop(['Date','Future Prediction'],1))[-Future_Prediciton:]

In [18]:
# Finally let's look at the predictions for future prices

#Shiba
shiba_prediciton = shiba_lr.predict(shiba_forecast)

np.set_printoptions(formatter={'float_kind':'{:f}'.format})
print(shiba_prediciton)

[[0.000025]
 [0.000025]
 [0.000026]
 [0.000026]
 [0.000025]
 [0.000025]
 [0.000025]
 [0.000026]
 [0.000025]
 [0.000025]]


In [19]:
#Doge

doge_prediction = doge_lr.predict(doge_forecast)

print(doge_prediction)

[[0.132423]
 [0.140803]
 [0.138841]
 [0.140555]
 [0.139364]
 [0.134274]
 [0.137573]
 [0.135302]
 [0.142430]
 [0.144451]]
